# Fine-Tuning a BERT Model and Create a Text Classifier

In the previous section, we've already performed the Feature Engineering to create BERT embeddings from the `reviews_body` text using the pre-trained BERT model, and split the dataset into train, validation and test files. To optimize for Tensorflow training, we saved the files in TFRecord format. 

Now, let’s fine-tune the BERT model to our Customer Reviews Dataset and add a new classification layer to predict the `star_rating` for a given `review_body`.

![BERT Training](img/bert_training.png)

As mentioned earlier, BERT’s attention mechanism is called a Transformer. This is, not coincidentally, the name of the popular BERT Python library, “Transformers,” maintained by a company called HuggingFace. 

We will use a variant of BERT called [**DistilBert**](https://arxiv.org/pdf/1910.01108.pdf) which requires less memory and compute, but maintains very good accuracy on our dataset.

In [30]:
import time
import random
import pandas as pd
from glob import glob
import argparse
import json
import subprocess
import sys
import os
import tensorflow as tf
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers.configuration_distilbert import DistilBertConfig

In [31]:
%store -r max_seq_length

no stored variable or alias max_seq_length


In [32]:
max_seq_length=64

In [33]:
try:
    max_seq_length
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the PREPARE section before you continue.')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [34]:
print(max_seq_length)

64


In [44]:
file = './data-pipeline/data.csv'

In [59]:
from pathlib import Path

filename_without_extension = Path(Path(file).stem).stem

In [60]:
import csv
df = pd.read_csv(file, 
                 delimiter=',')

In [61]:
print('Shape of dataframe {}'.format(df.shape))


Shape of dataframe (31005, 2)


# TODO:  Make sure our pipeline up to now (ie. SM Data Wrangler) outputs the correct target label (ie. 0-4, 1-5, -1/0/1)

In [79]:
df_train=df
df_validation=df
df_test=df

In [80]:
print(df_train)

       star_rating                                        review_body
0                4          Excellent. Looking forward to Windows 10.
1                3  I have used various versions over Norton over ...
2                1  I do not like the new budget outline it's very...
3                1  Quicken 2015 release 8 will not recognize or r...
4                4  These dopes think I'm going to write something...
...            ...                                                ...
31000            4                      Quicken 2015 works very well.
31001            3  Good cleaner software for my laptop, works bet...
31002            2  I am  not real happy with what I have. I keep ...
31003            3  I have been satisfied with the performance of ...
31004            1  I've used Quicken for years and I'm so terribl...

[31005 rows x 2 columns]


In [81]:
print(df_validation)

       star_rating                                        review_body
0                4          Excellent. Looking forward to Windows 10.
1                3  I have used various versions over Norton over ...
2                1  I do not like the new budget outline it's very...
3                1  Quicken 2015 release 8 will not recognize or r...
4                4  These dopes think I'm going to write something...
...            ...                                                ...
31000            4                      Quicken 2015 works very well.
31001            3  Good cleaner software for my laptop, works bet...
31002            2  I am  not real happy with what I have. I keep ...
31003            3  I have been satisfied with the performance of ...
31004            1  I've used Quicken for years and I'm so terribl...

[31005 rows x 2 columns]


In [82]:
print(df_test)

       star_rating                                        review_body
0                4          Excellent. Looking forward to Windows 10.
1                3  I have used various versions over Norton over ...
2                1  I do not like the new budget outline it's very...
3                1  Quicken 2015 release 8 will not recognize or r...
4                4  These dopes think I'm going to write something...
...            ...                                                ...
31000            4                      Quicken 2015 works very well.
31001            3  Good cleaner software for my laptop, works bet...
31002            2  I am  not real happy with what I have. I keep ...
31003            3  I have been satisfied with the performance of ...
31004            1  I've used Quicken for years and I'm so terribl...

[31005 rows x 2 columns]


In [83]:
train_inputs = df_train.apply(lambda x: Input(text = x[DATA_COLUMN], 
                                              label = x[LABEL_COLUMN]), axis = 1)

In [84]:
validation_inputs = df_validation.apply(lambda x: Input(text = x[DATA_COLUMN], 
                                                        label = x[LABEL_COLUMN]), axis = 1)



In [85]:
test_inputs = df_test.apply(lambda x: Input(text = x[DATA_COLUMN], 
                                            label = x[LABEL_COLUMN]), axis = 1)

In [86]:
tfrecord_output_path='./output'

In [87]:
import collections


In [88]:
train_data = '{}/bert/train'.format(tfrecord_output_path)
validation_data = '{}/bert/validation'.format(tfrecord_output_path)
test_data = '{}/bert/test'.format(tfrecord_output_path)

# Convert our train and validation features to InputFeatures (.tfrecord protobuf) that works with BERT and TensorFlow.
df_train_embeddings = convert_features_to_tfrecord(train_inputs, 
                                                   '{}/part-{}-{}.tfrecord'.format(train_data, 'algo-1', filename_without_extension), 
                                                   max_seq_length)

df_validation_embeddings = convert_features_to_tfrecord(validation_inputs, 
                                                        '{}/part-{}-{}.tfrecord'.format(validation_data, 'algo-1', filename_without_extension), 
                                                        max_seq_length)

df_test_embeddings = convert_features_to_tfrecord(test_inputs, 
                                                  '{}/part-{}-{}.tfrecord'.format(test_data, 'algo-1', filename_without_extension), 
                                                  max_seq_length)

Writing example 0 of 31005
Writing example 1000 of 31005
Writing example 2000 of 31005
Writing example 3000 of 31005
Writing example 4000 of 31005
Writing example 5000 of 31005
Writing example 6000 of 31005
Writing example 7000 of 31005
Writing example 8000 of 31005
Writing example 9000 of 31005
Writing example 10000 of 31005
Writing example 11000 of 31005
Writing example 12000 of 31005
Writing example 13000 of 31005
Writing example 14000 of 31005
Writing example 15000 of 31005
Writing example 16000 of 31005
Writing example 17000 of 31005
Writing example 18000 of 31005
Writing example 19000 of 31005
Writing example 20000 of 31005
Writing example 21000 of 31005
Writing example 22000 of 31005
Writing example 23000 of 31005
Writing example 24000 of 31005
Writing example 25000 of 31005
Writing example 26000 of 31005
Writing example 27000 of 31005
Writing example 28000 of 31005
Writing example 29000 of 31005
Writing example 30000 of 31005
Writing example 31000 of 31005
Writing example 0 of 

In [90]:
def select_data_and_label_from_record(record):
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids']
    }
    y = record['label_ids']

    return (x, y)

In [92]:
def file_based_input_dataset_builder(channel,
                                     input_filenames,
                                     pipe_mode,
                                     is_training,
                                      drop_remainder):

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.

    if pipe_mode:
        print('***** Using pipe_mode with channel {}'.format(channel))
        from sagemaker_tensorflow import PipeModeDataset
        dataset = PipeModeDataset(channel=channel,
                                  record_format='TFRecord')
    else:
        print('***** Using input_filenames {}'.format(input_filenames))
        dataset = tf.data.TFRecordDataset(input_filenames)

    dataset = dataset.repeat(100)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    name_to_features = {
      "input_ids": tf.io.FixedLenFeature([max_seq_length], tf.int64),
      "input_mask": tf.io.FixedLenFeature([max_seq_length], tf.int64),
      "segment_ids": tf.io.FixedLenFeature([max_seq_length], tf.int64),
      "label_ids": tf.io.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        return tf.io.parse_single_example(record, name_to_features)
        
    dataset = dataset.apply(
        tf.data.experimental.map_and_batch(
          lambda record: _decode_record(record, name_to_features),
          batch_size=8,
          drop_remainder=drop_remainder,
          num_parallel_calls=tf.data.experimental.AUTOTUNE))

    dataset.cache()

    if is_training:
        dataset = dataset.shuffle(seed=42,
                                  buffer_size=10,
                                  reshuffle_each_iteration=True)

    return dataset

In [94]:
#train_data = './data-tfrecord/bert-train'

train_data = './output/bert/train'

train_data_filenames = glob('{}/*.tfrecord'.format(train_data))
print('train_data_filenames {}'.format(train_data_filenames))

train_dataset = file_based_input_dataset_builder(
    channel='train',
    input_filenames=train_data_filenames,
    pipe_mode=False,
    is_training=True,
    drop_remainder=False).map(select_data_and_label_from_record)

train_data_filenames ['./output/bert/train/part-algo-1-data.tfrecord']
***** Using input_filenames ['./output/bert/train/part-algo-1-data.tfrecord']


In [95]:
# validation_data = './data-tfrecord/bert-validation'

validation_data = './output/bert/validation'
validation_data_filenames = glob('{}/*.tfrecord'.format(validation_data))
print('validation_data_filenames {}'.format(validation_data_filenames))

validation_dataset = file_based_input_dataset_builder(
    channel='validation',
    input_filenames=validation_data_filenames,
    pipe_mode=False,
    is_training=False,
    drop_remainder=False).map(select_data_and_label_from_record)

validation_data_filenames ['./output/bert/validation/part-algo-1-data.tfrecord']
***** Using input_filenames ['./output/bert/validation/part-algo-1-data.tfrecord']


In [96]:
#test_data = './data-tfrecord/bert-test'

test_data = './output/bert/test'
test_data_filenames = glob('{}/*.tfrecord'.format(test_data))
print(test_data_filenames)

test_dataset = file_based_input_dataset_builder(
    channel='test',
    input_filenames=test_data_filenames,
    pipe_mode=False,
    is_training=False,
    drop_remainder=False).map(select_data_and_label_from_record)

['./output/bert/test/part-algo-1-data.tfrecord']
***** Using input_filenames ['./output/bert/test/part-algo-1-data.tfrecord']


# Specify Manual Hyper-Parameters

In [110]:
epochs=1
steps_per_epoch=5
validation_steps=5
test_steps=5
freeze_bert_layer=True
learning_rate=3e-5
epsilon=1e-08

# Load Pretrained BERT Model 
https://huggingface.co/transformers/pretrained_models.html 

In [ ]:
CLASSES=[1, 2, 3, 4, 5]

config = DistilBertConfig.from_pretrained('distilbert-base-uncased',
                                          num_labels=len(CLASSES))

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', 
                                                              config=config)

# Setup the Custom Classifier Model Here

In [ ]:
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric=tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon)
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
model.layers[0].trainable=not freeze_bert_layer
model.summary()

In [105]:
callbacks = []

log_dir = './tmp/tensorboard/'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callbacks.append(tensorboard_callback)

In [109]:
history = model.fit(train_dataset,
                    shuffle=True,
                    epochs=epochs,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=validation_dataset,
                    validation_steps=validation_steps,
                    callbacks=callbacks)

 5/10 [==============>...............] - ETA: 8s - loss: 1.5558 - accuracy: 0.3500 

ResourceExhaustedError:  OOM when allocating tensor with shape[30522,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node Adam/Adam/update/AssignVariableOp (defined at <ipython-input-106-5f1fb29ad877>:7) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_27072]

Function call stack:
train_function


In [ ]:
print('Trained model {}'.format(model))

# Evaluate on Holdout Test Dataset

In [ ]:
test_history = model.evaluate(test_dataset,
                              steps=test_steps,                            
                              callbacks=callbacks)
print(test_history)

# Save the Model

In [ ]:
model_dir = './tmp/fine-tuned'

In [ ]:
!mkdir -p $model_dir

In [ ]:
model.save_pretrained(model_dir)

In [ ]:
!ls -al $model_dir

In [ ]:
cat $model_dir/config.json

# Predict with Model

In [ ]:
import json
from transformers import TFDistilBertForSequenceClassification

loaded_model = TFDistilBertForSequenceClassification.from_pretrained(model_dir,
                                                                     id2label={
                                                                       0: 1,
                                                                       1: 2,
                                                                       2: 3,
                                                                       3: 4,
                                                                       4: 5
                                                                     },
                                                                     label2id={
                                                                       1: 0,
                                                                       2: 1,
                                                                       3: 2,
                                                                       4: 3,
                                                                       5: 4
                                                                     })

In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
from transformers import TextClassificationPipeline
inference_pipeline = TextClassificationPipeline(model=loaded_model, 
                                                tokenizer=tokenizer,
                                                framework='tf',
                                                device=-1) # -1 is CPU, >= 0 is GPU

print("""I loved it!  I will recommend this to everyone.""", inference_pipeline("""I loved it!  I will recommend this to everyone."""))
print("""Really bad.  I hope they don't make this anymore.""", inference_pipeline("""Really bad.  I hope they don't make this anymore."""))

# Test Model

In [ ]:
import csv

df_sample_reviews = pd.read_csv('./data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz', 
                                delimiter='\t', 
                                quoting=csv.QUOTE_NONE,
                                compression='gzip')[['review_body', 'star_rating']].sample(n=100)
df_sample_reviews.shape

In [ ]:
df_sample_reviews.head()

In [ ]:
import pandas as pd

def predict(review_body):
    prediction_map = inference_pipeline(review_body)
    return prediction_map[0]['label']
    
y_pred = df_sample_reviews['review_body'].map(predict)

y_pred

In [ ]:
y_true = df_sample_reviews['star_rating']

y_true

# Classification Report

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_true, y_pred=y_pred))

# Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

print('Test Accuracy: ', accuracy_score(y_pred=y_pred, y_true=y_true))

# Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

def plot_conf_mat(cm, classes, title, cmap = plt.cm.Greens):
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
        horizontalalignment="center",
        color="black" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

In [ ]:
import itertools
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

cm = confusion_matrix(y_true=y_true, y_pred=y_pred)

plt.figure()
fig, ax = plt.subplots(figsize=(10,5))
plot_conf_mat(cm, 
              classes=['1', '2', '3', '4', '5'], 
              title='Confusion Matrix')
plt.show()

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();